In [1]:
from data.process import *

2024-07-27 17:13:15.841426: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-27 17:13:15.851010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 17:13:15.868019: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 17:13:15.868045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 17:13:15.882775: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after first optimization
    'colsample_bytree': 0.25,
    'learning_rate': 0.05,

    # fixed after third optimization
    'max_depth': 128,
    'num_leaves': 4096,
    'min_data_in_leaf': 25,

    # override fixed params
    'n_estimators': 1000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['category']).columns)

In [6]:
%%time
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.887088	valid_0's binary_logloss: 0.250385
[200]	valid_0's auc: 0.890436	valid_0's binary_logloss: 0.243352
[300]	valid_0's auc: 0.891593	valid_0's binary_logloss: 0.241792
[400]	valid_0's auc: 0.892032	valid_0's binary_logloss: 0.241305
[500]	valid_0's auc: 0.892241	valid_0's binary_logloss: 0.241075
[600]	valid_0's auc: 0.892353	valid_0's binary_logloss: 0.240949
[700]	valid_0's auc: 0.892403	valid_0's binary_logloss: 0.24092
[800]	valid_0's auc: 0.892462	valid_0's binary_logloss: 0.240884
[900]	valid_0's auc: 0.892513	valid_0's binary_logloss: 0.24084
[1000]	valid_0's auc: 0.892471	valid_0's binary_logloss: 0.240902
CPU times: user 4h 23min 18s, sys: 9.59 s, total: 4h 23min 28s
Wall time: 11min 5s


In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.8925317511260051),
                                     ('binary_logloss', 0.24082650440346098)])})


In [8]:
%%time
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]

CPU times: user 44min 40s, sys: 908 ms, total: 44min 41s
Wall time: 2min 4s
